In [9]:
import json
import base64
import struct
import random

def read_file_and_convert_to_gltf(input_file, output_file):
    # Read input file
    with open(input_file, 'r') as f:
        lines = f.readlines()

    # Parse data
    vertices = []
    for line in lines[1:]:  # Skip header line
        parts = line.strip().split()
        if len(parts) == 3:
            z, x, y = map(float, parts)
            vertices.append([x, y, z])

    # Generate random colors for each vertex
    colors = [[0,0,1] for _ in range(len(vertices))]

    # GLTF structure
    gltf_data = {
        "asset": {
            "version": "2.0"
        },
        "scenes": [{
            "nodes": [0]
        }],
        "nodes": [{
            "mesh": 0
        }],
        "meshes": [{
            "primitives": [{
                "attributes": {
                    "POSITION": 0,
                    "COLOR_0": 1
                },
                "mode": 0
            }]
        }],
        "accessors": [{
            "bufferView": 0,
            "componentType": 5126,  # FLOAT
            "count": len(vertices),
            "type": "VEC3",
            "min": [min(v[0] for v in vertices), min(v[1] for v in vertices), min(v[2] for v in vertices)],
            "max": [max(v[0] for v in vertices), max(v[1] for v in vertices), max(v[2] for v in vertices)]
        }, {
            "bufferView": 1,
            "componentType": 5126,  # FLOAT
            "count": len(colors),
            "type": "VEC3"
        }],
        "bufferViews": [{
            "buffer": 0,
            "byteOffset": 0,
            "byteLength": len(vertices) * 12,  # 3 floats per vertex, 4 bytes each
            "target": 34962  # ARRAY_BUFFER
        }, {
            "buffer": 1,
            "byteOffset": 0,
            "byteLength": len(colors) * 12,  # 3 floats per color, 4 bytes each
            "target": 34962  # ARRAY_BUFFER
        }],
        "buffers": [{
            "byteLength": len(vertices) * 12,  # 3 floats per vertex, 4 bytes each
            "uri": "data:application/octet-stream;base64,"
        }, {
            "byteLength": len(colors) * 12,  # 3 floats per color, 4 bytes each
            "uri": "data:application/octet-stream;base64,"
        }]
    }

    # Serialize vertices
    vertex_buffer_data = bytearray()
    for vertex in vertices:
        vertex_buffer_data += bytearray(struct.pack('<3f', *vertex))

    # Serialize colors
    color_buffer_data = bytearray()
    for color in colors:
        color_buffer_data += bytearray(struct.pack('<3f', *color))

    # Encode binary data as base64
    base64_vertex_data = base64.b64encode(vertex_buffer_data).decode()
    base64_color_data = base64.b64encode(color_buffer_data).decode()

    # Update GLTF buffer URIs with base64 data
    gltf_data['buffers'][0]['uri'] += base64_vertex_data
    gltf_data['buffers'][1]['uri'] += base64_color_data

    # Write GLTF to file
    with open(output_file, 'w') as f:
        json.dump(gltf_data, f)

# Example usage:
read_file_and_convert_to_gltf("./alti2.xyz", "output.gltf")


In [31]:
import json
import base64
import struct
import rasterio

def read_file_and_convert_to_gltf(input_xyz_file, input_tif_file, output_gltf_file):
    # Read XYZ file
    with open(input_xyz_file, 'r') as f:
        lines = f.readlines()

    # Parse XYZ data
    vertices = []
    for line in lines[1:]:  # Skip header line
        parts = line.strip().split()
        if len(parts) == 3:
            x, y, z = map(float, parts)
            vertices.append((z, x, y))

    # Extract RGB values for each point from GeoTIFF
    rgb_values = []
    with rasterio.open(input_tif_file) as src:
        for vertex in vertices:
            z, x, y = vertex
            row, col = src.index(x, y)
            if 0 <= row < src.height and 0 <= col < src.width:
                rgb = src.read(1, window=((row, row+1), (col, col+1)))
                rgb_values.append([int(rgb[0]), int(rgb[0]), int(rgb[0])])  # Take the RGB values from the pixel
            else:
                print(f"Point ({x}, {y}) is outside the bounds of the GeoTIFF image.")
    rgb_values = [[0,0,1] for _ in range(len(vertices))]

    # Restlicher Code bleibt unverändert...



    # GLTF structure
    gltf_data = {
        "asset": {
            "version": "2.0"
        },
        "scenes": [{
            "nodes": [0]
        }],
        "nodes": [{
            "mesh": 0
        }],
        "meshes": [{
            "primitives": [{
                "attributes": {
                    "POSITION": 0,
                    "COLOR_0": 1
                },
                "mode": 0
            }]
        }],
        "accessors": [{
            "bufferView": 0,
            "componentType": 5126,  # FLOAT
            "count": len(vertices),
            "type": "VEC3",
            "min": [min(v[0] for v in vertices), min(v[1] for v in vertices), min(v[2] for v in vertices)],
            "max": [max(v[0] for v in vertices), max(v[1] for v in vertices), max(v[2] for v in vertices)]
        }, {
            "bufferView": 1,
            "componentType": 5126,  # FLOAT
            "count": len(rgb_values),
            "type": "VEC3"
        }],
        "bufferViews": [{
            "buffer": 0,
            "byteOffset": 0,
            "byteLength": len(vertices) * 12,  # 3 floats per vertex, 4 bytes each
            "target": 34962  # ARRAY_BUFFER
        }, {
            "buffer": 1,
            "byteOffset": 0,
            "byteLength": len(rgb_values) * 12,  # 3 floats per color, 4 bytes each
            "target": 34962  # ARRAY_BUFFER
        }],
        "buffers": [{
            "byteLength": len(vertices) * 12,  # 3 floats per vertex, 4 bytes each
            "uri": "data:application/octet-stream;base64,"
        }, {
            "byteLength": len(rgb_values) * 12,  # 3 floats per color, 4 bytes each
            "uri": "data:application/octet-stream;base64,"
        }]
    }

    # Serialize vertices
    vertex_buffer_data = bytearray()
    for vertex in vertices:
        vertex_buffer_data += bytearray(struct.pack('<3f', *vertex))

    # Serialize colors (RGB values)
    color_buffer_data = bytearray()
    for rgb in rgb_values:
        color_buffer_data += bytearray(struct.pack('<3B', *rgb))

    # Encode binary data as base64
    base64_vertex_data = base64.b64encode(vertex_buffer_data).decode()
    base64_color_data = base64.b64encode(color_buffer_data).decode()

    # Update GLTF buffer URIs with base64 data
    gltf_data['buffers'][0]['uri'] += base64_vertex_data
    gltf_data['buffers'][1]['uri'] += base64_color_data

    # Write GLTF to file
    with open(output_gltf_file, 'w') as f:
        json.dump(gltf_data, f)

# Example usage:
read_file_and_convert_to_gltf("alti2.xyz", "image.tif", "output.gltf")


C:\Users\fruef\AppData\Local\Temp\ipykernel_16028\837188083.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rgb_values.append([int(rgb[0]), int(rgb[0]), int(rgb[0])])  # Take the RGB values from the pixel


In [51]:
import json
import base64
import struct
import rasterio

def read_file_and_convert_to_gltf(input_xyz_file, input_tif_file, output_gltf_file):
    # Read input file
    with open(input_xyz_file, 'r') as f:
        lines = f.readlines()

    # Parse data
    vertices = []
    for line in lines[1:]:  # Skip header line
        parts = line.strip().split()
        if len(parts) == 3:
            z, x, y = map(float, parts)
            vertices.append([x, y, z])

    # Generate random colors for each vertex
    colors =  []
    with rasterio.open(input_tif_file) as src:
        for vertex in vertices:
            y, z, x = vertex
            row, col = src.index(x, y)
            if 0 <= row < src.height and 0 <= col < src.width:
                g = src.read(1, window=((row, row+1), (col, col+1)))
                # b = src.read(2, window=((row, row+1), (col, col+1)))
                # r = src.read(3, window=((row, row+1), (col, col+1)))
                colors.append([int(g[0]), int(g[0]), int(g[0])])  # Take the RGB values from the pixel
            else:
                print(f"Point ({x}, {y}) is outside the bounds of the GeoTIFF image.")

    # GLTF structure
    gltf_data = {
        "asset": {
            "version": "2.0"
        },
        "scenes": [{
            "nodes": [0]
        }],
        "nodes": [{
            "mesh": 0
        }],
        "meshes": [{
            "primitives": [{
                "attributes": {
                    "POSITION": 0,
                    "COLOR_0": 1
                },
                "mode": 0
            }]
        }],
        "accessors": [{
            "bufferView": 0,
            "componentType": 5126,  # FLOAT
            "count": len(vertices),
            "type": "VEC3",
            "min": [min(v[0] for v in vertices), min(v[1] for v in vertices), min(v[2] for v in vertices)],
            "max": [max(v[0] for v in vertices), max(v[1] for v in vertices), max(v[2] for v in vertices)]
        }, {
            "bufferView": 1,
            "componentType": 5126,  # FLOAT
            "count": len(colors),
            "type": "VEC3"
        }],
        "bufferViews": [{
            "buffer": 0,
            "byteOffset": 0,
            "byteLength": len(vertices) * 12,  # 3 floats per vertex, 4 bytes each
            "target": 34962  # ARRAY_BUFFER
        }, {
            "buffer": 1,
            "byteOffset": 0,
            "byteLength": len(colors) * 12,  # 3 floats per color, 4 bytes each
            "target": 34962  # ARRAY_BUFFER
        }],
        "buffers": [{
            "byteLength": len(vertices) * 12,  # 3 floats per vertex, 4 bytes each
            "uri": "data:application/octet-stream;base64,"
        }, {
            "byteLength": len(colors) * 12,  # 3 floats per color, 4 bytes each
            "uri": "data:application/octet-stream;base64,"
        }]
    }

    # Serialize vertices
    vertex_buffer_data = bytearray()
    for vertex in vertices:
        vertex_buffer_data += bytearray(struct.pack('<3f', *vertex))

    # Serialize colors
    color_buffer_data = bytearray()
    for color in colors:
        color_buffer_data += bytearray(struct.pack('<3f', *color))

    # Encode binary data as base64
    base64_vertex_data = base64.b64encode(vertex_buffer_data).decode()
    base64_color_data = base64.b64encode(color_buffer_data).decode()

    # Update GLTF buffer URIs with base64 data
    gltf_data['buffers'][0]['uri'] += base64_vertex_data
    gltf_data['buffers'][1]['uri'] += base64_color_data

    # Write GLTF to file
    with open(output_gltf_file, 'w') as f:
        json.dump(gltf_data, f)

# Example usage:

read_file_and_convert_to_gltf("alti.xyz", "image.tif", "output.gltf")


C:\Users\fruef\AppData\Local\Temp\ipykernel_16028\2059834308.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  colors.append([int(g[0]), int(g[0]), int(g[0])])  # Take the RGB values from the pixel
